In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00


In [2]:
import optuna
import time

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten , Dropout
from tensorflow import keras

#### 데이터 불러오기

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_y),(test_X, test_y) = fashion_mnist.load_data()

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
# 전처리

train_X = train_X/255.0
test_X = test_X /255.0

train_X = train_X.reshape(-1 ,28,28,1)  # 3d ---> 4d
test_X  = test_X.reshape(-1, 28,28,1)

print(train_X.shape)
print(train_X[0].shape)

(60000, 28, 28, 1)
(28, 28, 1)


#### 모델 구조 설계

In [6]:
model_vgg_fashion = tf.keras.Sequential(
    [
        # 원본 입력을 기준으로 설계.... : 224 224 RGB
        # --> 28 28 1
        Conv2D( input_shape=(28,28,1), kernel_size=(3,3),
               filters=64, padding ="same", activation="relu"),

        Conv2D(  kernel_size=(3,3),
               filters=64, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),



        Conv2D(  kernel_size=(3,3),
               filters=128, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=128, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),



        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),



        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),
        # ===> 기본적인 이미지가 가지고 있는 특징 추출!!!!!

        # 분류를 위한 NN
        # 1) 특징을 분류 Faltten
        Flatten(),


        # 2) 분류용 HL 적층!!!!
        Dense( units=4096, activation="relu"),
        Dense( units=4096, activation="relu"),


        # 3) 출력용 --> 내 모델의 목적에 맞춰야함!!
        #    나는 10개만 하면 됨...수정..
        Dense( units=10, activation="softmax")
    ]
)
model_vgg_fashion

In [ ]:
# 위의 간단한 구조들을 바탕으로 좀 더 디테일한 구조들에 대해서 실험
# 1. 내가 무엇을 보고 최적화를 할지 함수로 설정
# ===> 전에 ML을 할 때는 단순 파라미터들의 조합
# ===> DL : 모델의 구조들이 기본적인 파라미터
#      모델의 구조들에 대한 실험을 함수로 구현을 해야함!!!!
#      ++ 노드의 수 ++ lr ++ AF etc
# :: 내가 하려고 하는 목적에 맞는 함수를 직접 만드셔야 함!!!

In [ ]:
# Step1) optuna에서 주어진 파라미터에 맞는 모델 구조!!!! 함수
#  ---> 입력
#       1) 1번 conv layer에서 몇 개의 Conv을 진행할지..(2,3)
#       2) 2번 conv layer에서 몇 개의 conv를 진행할지
#       3) dropout 같은 구조도 하고 싶으면 rate테스트
#       4) 분류하는 곳에서 dense의 node 수를 조절하고

# 내가 하고자 하는 구조에 대한 생성함수
# (optuna가 찾는 목적함수와 다른 함수)

# optuna의 objectitve Function이 제시하는 값에 따른 구조를 생성하는 함수

In [ ]:
def create_model( num_layers_conv1, num_filters_conv1,
                  num_layers_conv2, num_filters_conv2,
                  dnn_dropout_rate, dnn_num_node) :
# 여기에서는 모델을 설계하는 방식 중
# 기존의 리스트에 레이어를 적층하는 구조 방식이라
# 리스트의 append를 이용해서 직접 추가하도록 하겠음

  model = tf.keras.Sequential()

  # 1번 conv
  model.add( Conv2D (input_shape = (28,28,1),
                     kernel_size = (2,2), filters = 32, padding = "same", activation ="relu")

  )

  for i in range(num_layers_conv1):
    model.add(Conv2D(input_shape = (28,28,1),
                       kernel_size = (2,2), filters = num_filters_conv1, padding = "same", activation ="relu"))

  model.add( MaxPool2D( pool_size=(2,2), strides=(2,2)) )


  # 2번 conv
  model.add( Conv2D (kernel_size = (2,2), filters = 32, padding = "same", activation ="relu"))

  for i in range(num_layers_conv2) :
    model.add( Conv2D (input_shape = (28,28,1),
                     kernel_size = (2,2), filters = num_filters_conv2, padding = "same", activation ="relu"))

  model.add( MaxPool2D( pool_size=(2,2), strides=(2,2)) )


# 분류를 위한 모델 셜계
  model.add(Flatten())
  model.add(Dense(units = dnn_num_node, activation ="relu"))
  model.add(Dropout(rate = dnn_dropout_rate))
  model.add(Dense(units = dnn_num_node, activation ="relu"))
  model.add(Dropout(rate = dnn_dropout_rate))

# 출력층을 설계
  model.add(Dense( units=10, activation="softmax"))
  return model


In [7]:
# Step2) 최적의 값을 찾는 objective function 설계!!!!
# ===> 앞에서 ML했던 방식 그대로...
def object_vgg(trail):
    # 혹시 뒤에서 keras 백단에서 살아있을 때...
    # ===> 클리어해서 정리하고 시작하자!!!!
  keras.backend.clear_session()

  # 내가 테스트할 파라미터들을 제안 받는 것들 세팅!!!!!!!!
  num_layers_conv1 = trail.suggest_int("num_layers_conv1",1,4)
  num_filters_conv1 =trail.suggest_int("num_filters_conv1",32,128)
  num_layers_conv2 = trail.suggest_int("num_layers_conv2",1,4)
  num_filters_conv2 = trail.suggest_int("num_filters_conv2",182,256)
  dnn_dropout_rate = trail.suggest_float("dnn_dropout_rate", 0.1, 0.4)
  dnn_num_node =  trail.suggest_int("dnn_num_node", 128, 2048)
# -> 여기까지 모델 설계도에 관련된 파라미터

# + 모델의 학습하는 영향을 미치는 파라미터도 중요
# -> 여기서는 간단하게 최적화 방법만 선택

# learning 과 관련된 HPT
# + lr 학습에서 테스트도 하는 것도 중요함
# + batch_size etc 테스트의 영역

  optimizer = trail.suggest_categorical("optimizer",["adam", "sgd"])

  model  = create_model( num_layers_conv1, num_filters_conv1,
                        num_layers_conv2, num_filters_conv2,
                        dnn_dropout_rate, dnn_num_node)



  model.compile(
      optimizer = optimizer,
      loss = "sparse_categorical_crossentropy",
      metrics = ["accuracy"]
  )
  # 제안 받은 모델 구조도 + 학습 방식에 것을 적용!
  # + 시간 관계상 확 줄여서 진행하겠음.
  # callback( early stopping을 주로 같이 )


  history = model.fit( train_X , train_y,
                      epochs = 2, batch_size = 1024,
                       validation_split = 0.25)
   # objective function의 타겟
   # -> 주관적으로 val_accuracy를 타겟으로 하겠음



  score = history.history["val_accuracy"][-1]


  return score


In [8]:

# 실제 test
study_my_vgg = optuna.create_study(direction = "maximize")

# 원래는 엄청 시도해야하는데 시도 횟수도 줄여서
study_my_vgg.optimize(object_vgg, n_trials = 2, n_jobs= -1)

[I 2024-06-06 14:26:00,401] A new study created in memory with name: no-name-433c59f4-bdc6-4249-b6c4-9da5f92c72d3
[W 2024-06-06 14:26:00,412] Trial 0 failed with parameters: {'num_layers_conv1': 4, 'num_filters_conv1': 126, 'num_layers_conv2': 3, 'num_filters_conv2': 198, 'dnn_dropout_rate': 0.2576763241781195, 'dnn_num_node': 331, 'optimizer': 'adam'} because of the following error: NameError("name 'create_model' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-7-1a299d5f4c34>", line 26, in object_vgg
    model  = create_model( num_layers_conv1, num_filters_conv1,
NameError: name 'create_model' is not defined
[W 2024-06-06 14:26:00,417] Trial 0 failed with value None.
[W 2024-06-06 14:26:00,416] Trial 1 failed with parameters: {'num_layers_conv1': 4, 'num_filters_conv1': 128, 'num_layers_conv2': 1, 'num_filters_conv2': 248, 'd